In [33]:
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
nlp = spacy.load('en_core_web_sm')
text = "The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Because it has achieved significance within the past fifty years, Criteria Consideration G applies. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions. She had the honor of being chosen as the Return to Flight vehicle after both the Challenger and Columbia accidents. Discovery was the first shuttle to fly with the redesigned SRBs, a result of the Challenger accident, and the first shuttle to fly with the Phase II and Block I SSME. Discovery also carried the Hubble Space Telescope to orbit and performed two of the five servicing missions to the observatory. She flew the first and last dedicated Department of Defense (DoD) missions, as well as the first unclassified defense-related mission. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. She was the first orbiter to dock to the ISS, and the first to perform an exchange of a resident crew. Under Criterion C, Discovery is significant as a feat of engineering. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” Although her base structure followed a conventional aircraft design, she used advanced materials that both minimized her weight for cargo-carrying purposes and featured low thermal expansion ratios, which provided a stable base for her Thermal Protection System (TPS) materials. The Space Shuttle orbiter also featured the first reusable TPS; all previous spaceflight vehicles had a single-use, ablative heat shield. Other notable engineering achievements of the orbiter included the first reusable orbital propulsion system, and the first two-fault-tolerant Integrated Avionics System. As Hale stated, the Space Shuttle remains “the largest, fastest, winged hypersonic aircraft in history,” having regularly flown at twenty-five times the speed of sound."

doc = nlp(text)

sentences = [sent.text for sent in doc.sents]
for sentence in sentences[:3]:
    print(sentence)

The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering.
Because it has achieved significance within the past fifty years, Criteria Consideration G applies.
Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.


In [39]:
import string
from collections import Counter

# Удаляем стоп-слова и пунктуацию
stop_words = nlp.Defaults.stop_words  # Список стоп-слов SpaCy
punctuation = string.punctuation + '\n'

# Подсчет частоты слов
word_frequencies = Counter(
    token.text.lower() for token in doc
    if token.text.lower() not in stop_words and token.text not in punctuation
)

# Нормализ частоты
max_frequency = max(word_frequencies.values())
for word in word_frequencies:
    word_frequencies[word] /= max_frequency
# Топ-10 самых частых слов
print(word_frequencies.most_common(10))


[('space', 1.0), ('shuttle', 0.6153846153846154), ('orbiter', 0.5384615384615384), ('discovery', 0.5384615384615384), ('criterion', 0.3076923076923077), ('missions', 0.3076923076923077), ('program', 0.23076923076923078), ('engineering', 0.23076923076923078), ('station', 0.23076923076923078), ('flew', 0.23076923076923078)]


In [40]:
# Оценка значимости предложений
sentence_scores = {}
for sent in doc.sents:
    for token in sent:
        if token.text.lower() in word_frequencies:
            if sent not in sentence_scores:
                sentence_scores[sent] = word_frequencies[token.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[token.text.lower()]
# Посмотр на значимость каждого предложения
print(sentence_scores)


{The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering.: 6.307692307692307, Because it has achieved significance within the past fifty years, Criteria Consideration G applies.: 0.6153846153846154, Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.: 5.769230769230768, Unlike the Mercury, Gemini, and Apollo programs, the SSP’s emphasis was on cost effectiveness and reusability, and eventually the construction of a space station.: 2.3076923076923075, Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than 

In [41]:
from heapq import nlargest

# Выбop 30% самых значимых предложений
select_length = int(len([sent for sent in doc.sents]) * 0.3)
summary_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)

# Формир summary
summary = ' '.join([sent.text for sent in summary_sentences])
print(summary)


The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.” In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle. Including her maiden voyage

In [42]:
from IPython.display import Markdown

# Markdown-вывод
formatted_summary = "### Text Summary\n\n"
for i, sent in enumerate(summary_sentences, start=1):
    formatted_summary += f"{i}. {sent.text.strip()}\n\n"
display(Markdown(formatted_summary))


### Text Summary

1. The Orbiter Discovery, OV-103, is considered eligible for listing in the National Register of Historic Places (NRHP) in the context of the U.S. Space Shuttle Program (1969-2011) under Criterion A in the areas of Space Exploration and Transportation and under Criterion C in the area of Engineering.

2. Under Criterion A, Discovery is significant as the oldest of the three extant orbiter vehicles constructed for the Space Shuttle Program (SSP), the longest running American space program to date; she was the third of five orbiters built by NASA.

3. According to Wayne Hale, a flight director from Johnson Space Center, the Space Shuttle orbiter represents a “huge technological leap from expendable rockets and capsules to a reusable, winged, hypersonic, cargo-carrying spacecraft.”

4. In addition, Discovery was vital to the construction of the International Space Station (ISS); she flew thirteen of the thirty-seven total missions flown to the station by a U.S. Space Shuttle.

5. Including her maiden voyage (launched August 30, 1984), Discovery flew to space thirty-nine times, more than any of the other four orbiters; she was also the first orbiter to fly twenty missions.



In [43]:
# Сохранение summary в текстовый файл
with open("summary.txt", "w") as file:
    file.write("=== Text Summary ===\n\n")
    for i, sent in enumerate(summary_sentences, start=1):
        file.write(f"{i}. {sent.text.strip()}\n")
    file.write("====================\n")

print("Summary saved to 'summary.txt'")


Summary saved to 'summary.txt'
